# Feature Engineering and Creation of flat file

After the exploratory data analysis done step1:(*Refer "1.Instacart-EDA.ipynb" *)
In this notebook ,I am creating derieved features (user based features, product based features and user-product paired features) and merging them together to create a flat file which will be further used for Logistic Regression

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
from collections import OrderedDict

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from sklearn import metrics 
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

### Load the files into Datframes and dataset description.

- **aisles_df** : contains aisle information . There are 134 rows and 2 columns. aisle id and its corresponding aisle name
- **products_df** : contains product information. 49688 rows and 4 columns. There are 49688 different products instacart offers its customers. This dataframe contains information inform of product id, product name, aisle id and department id.
- **orders_df:** contains order related information.3421083 rows and 7 columns. This dataframe contains order id, user id, eval set (prior, train, test),order number (for the specific user),day of week when order was placed,hour and how many days elapsed since last order.
This dataset describes 3421083 instacart orders. All these orders belong to 206209 users

- **order_products_prior_df**: contains every order and all the products belonging to an order. All these orders are belonging to prior set. 32434489 rows and 4 colums. It contains order id, product_id, add to cart order of that product in the specific order id, and whether it was a reordered product.
- **departments_df**: containd department information. 21 rows and 2 columns. There are 21 departments along with their department ids
- **order_products_train_df**: contains every order and all the products belonging to an order. All these orders are belonging to train set. 1038515 rows and 4 colums. It contains order id, product_id, add to cart order of that product in the specific order id, and whether it was a reordered product.All these training orders belong to 98406 users.

Currently the test dataframe (order_products__test_cap.csv) is not loaded here

In [2]:
#Import the files

aisles_df = pd.read_csv('aisles.csv')
products_df = pd.read_csv('products.csv')
orders_df = pd.read_csv('orders.csv')
order_products_prior_df = pd.read_csv('order_products__prior.csv')
departments_df = pd.read_csv('departments.csv')
order_products_train_df = pd.read_csv('order_products__train_cap.csv')

- Merging Order_product_train with orders_df to obtain order level details (*like userid,order number for the user,day of order, hour of order*) for the order- product pairings used for training

- Merging Order_product_prior with orders_df to obtain order level details (*like userid,which order number it was for the user,day of order, hour of order*) for the order- product pairings of prior orders of users.Currently this table has details for all users. Going forward this dataframe will be filtered to retain this detail only for users who are considered as training users. These training user ids are obtained from unique user ids of order_product train_df


In [3]:
#obtaining order level details

order_products_train_df = order_products_train_df.merge(orders_df.drop('eval_set', axis=1), on='order_id')
order_products_prior_df = order_products_prior_df.merge(orders_df.drop('eval_set', axis=1), on='order_id')

### Creating User-Product feature

Here I am creating a new dataframe called user_product_df which gives information about product and user pair. Following new features are created:

- **user_product_avg_add_to_cart_order**: For the given User-Product pair, this column tells the average add to cart order of the product for this user
- **user_product_total_orders**: For the given User-Product pair, this column tells ,how many times this product was ordered by this user
- **user_product_avg_days_since_prior_order** :For the given User-Product pair, this column tells , average number of days elapsed since last time this product was ordered by the user
- **user_product_avg_order_dow**: For the given User-Product pair, this column tells , average day of the week when the user orders this product
- **user_product_avg_order_hour_of_day** :For the given User-Product pair, this column tells , average hour of the day when the user orders this product.

This dataframe contains this information for all the 206209 users.

In [14]:
user_prod_features1 = ['user_product_avg_add_to_cart_order','user_product_total_orders','user_product_avg_days_since_prior_order',
                      'user_product_avg_order_dow','user_product_avg_order_hour_of_day']

user_product_df = (order_products_prior_df.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict([('add_to_cart_order','mean'),( 'order_id','count'),('days_since_prior_order','mean'),
                                                     ('order_dow','mean'),
                                                     ('order_hour_of_day','mean')])))
user_product_df.columns = ['product_id','user_id'] + user_prod_features1
user_product_df.head()

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day
0,1,138,3.0,2,11.5,6.0,14.0
1,1,709,20.0,1,6.0,0.0,21.0
2,1,764,10.5,2,9.0,3.5,15.0
3,1,777,7.0,1,26.0,1.0,7.0
4,1,825,2.0,1,30.0,2.0,14.0


Obtaining unique User ids(training) 

In [15]:
train_ids = order_products_train_df['user_id'].unique() 
train_ids.shape

(98406L,)

Filtering user_product_df only for training ids obtained previously to create df_X

In [16]:
df_X = user_product_df[user_product_df['user_id'].isin(train_ids)]
df_X.head(2)

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day
1,1,709,20.0,1,6.0,0.0,21.0
3,1,777,7.0,1,26.0,1.0,7.0


Here a dataframe called train_carts is created which tells what all products were ordered by the trainng users in their latest order which is also the training order.



In [11]:
train_carts = (order_products_train_df.groupby('user_id',as_index=False)
                                      .agg({'product_id':(lambda x: set(x))})
                                      .rename(columns={'product_id':'latest_cart'}))
train_carts.head(3)

,user_id,latest_cart
0,5,"{40706, 21413, 20843, 48204, 21616, 19057, 201..."
1,7,"{29894, 17638, 47272, 45066, 13198, 37999, 408..."
2,8,"{27104, 15937, 5539, 41540, 31717, 48230, 2224..."


Next df_X and train_carts are merged. This new dataset contains historical (prior)order info ( which product ids were ordered by the training user and how many times and also if they are present in their latest order. ) A new column called in_cart is present .This tells whether a prior product ordered by the user is also present in the current order.

Eg: User Id (709) has previously ordered product_id(1,196,223 etc) but these 3 products are not present for this user_id's (709)latest order.

In [17]:
df_X = df_X.merge(train_carts, on='user_id')
df_X['in_cart'] = (df_X.apply(lambda row: row['product_id'] in row['latest_cart'], axis=1).astype(int))


In [18]:
df_X.head(3)

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day,latest_cart,in_cart
0,1,709,20.0,1,6.0,0.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0
1,196,709,25.0,2,8.0,0.5,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0
2,223,709,34.0,1,NaN,1.0,19.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0


In [10]:
df_X['in_cart'].value_counts()

0    5726982
1     621926
Name: in_cart, dtype: int64

The dataset is having class imbalance .There are more instances of which products the customer will not reorder than the instances of which products the customer will order

### Creating Product Based Features

Here I am creating a new dataframe called prod_features_df. Following product based features are created.

- **product_total_orders**: How many times a given product has been ordered overall
- **product_avg_add_to_cart_order** :This tells the average add to cart order of the product
- **product_avg_order_dow**: This tells the average day of week when this product is ordered
- **product_avg_order_hour_of_day**: This tells the average hour of the day when this product is ordered the most
- **product_avg_days_since_prior_order** : This tells the average number of days elapsed since this product was last ordered

In [11]:
prod_features = ['product_total_orders','product_avg_add_to_cart_order','product_avg_order_dow', 'product_avg_order_hour_of_day', 'product_avg_days_since_prior_order']

prod_features_df = (order_products_prior_df.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean'),('order_dow','mean'),
                                      ('order_hour_of_day', 'mean'),
                                      ('days_since_prior_order', 'mean')])))
prod_features_df.columns = ['product_id'] + prod_features
prod_features_df.head()

,product_id,product_total_orders,product_avg_add_to_cart_order,product_avg_order_dow,product_avg_order_hour_of_day,product_avg_days_since_prior_order
0,1,1852,5.801836,2.776458,13.238121,10.432725
1,2,90,9.888889,2.922222,13.277778,10.482759
2,3,277,6.415162,2.736462,12.104693,10.565385
3,4,329,9.507599,2.683891,13.714286,14.686207
4,5,15,6.466667,2.733333,10.666667,12.428571


Merging df_X and prod_features_df

In [12]:
df_X = df_X.merge(prod_features_df, on='product_id')

df_X = df_X.dropna()


### Creating User Based Features
Creating a dataframe containng user features.(user_features_df)

- **user_total_orders**: Total number of orders placed by the user
- **user_avg_cartsize** : Average cart size of the user
- **user_total_products** : Total number of products ordered by the user
- **user_avg_days_since_prior_order**: Number of days elapsed between subsequent orders
- **user_avg_order_dow** : Average day of the week when user places order
- **user_avg_order_hour_of_day**: Average hour of the day when user places order


In [13]:
user_features = ['user_total_orders','user_avg_cartsize','user_total_products','user_avg_days_since_prior_order','user_avg_order_dow','user_avg_order_hour_of_day']

user_features_df = (order_products_prior_df.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id',['nunique', (lambda x: x.shape[0] / x.nunique())]),
                                                    ('product_id','nunique'),
                                                    ('days_since_prior_order','mean'),('order_dow','mean'),
                                                    ('order_hour_of_day','mean')])))

user_features_df.columns = ['user_id'] + user_features

Merging df_X with user_features_df

In [14]:
df_X = df_X.merge(user_features_df, on='user_id')
df_X = df_X.dropna()

In [15]:
df_X['user_product_order_freq'] = df_X['user_product_total_orders'] / df_X['user_total_orders'] 
df_X.head()

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day,latest_cart,in_cart,product_total_orders,...,product_avg_order_dow,product_avg_order_hour_of_day,product_avg_days_since_prior_order,user_total_orders,user_avg_cartsize,user_total_products,user_avg_days_since_prior_order,user_avg_order_dow,user_avg_order_hour_of_day,user_product_order_freq
0,1,709,20.0,1,6.0,0.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,1852,...,2.776458,13.238121,10.432725,5,38,135,7.070968,1.651042,19.333333,0.2
1,196,709,25.0,2,8.0,0.5,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,35791,...,2.898550,12.523959,11.971826,5,38,135,7.070968,1.651042,19.333333,0.4
2,587,709,25.0,1,6.0,0.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,36,...,2.916667,12.805556,12.558824,5,38,135,7.070968,1.651042,19.333333,0.2
3,955,709,30.0,1,8.0,0.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,197,...,2.522843,12.705584,12.191489,5,38,135,7.070968,1.651042,19.333333,0.2
4,1219,709,31.0,1,8.0,1.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,374,...,2.743316,13.451872,10.288462,5,38,135,7.070968,1.651042,19.333333,0.2


### Some more feature engineering
Apart from creating user based features, product based features and User-Product pair based features,5 more features are created which tells how different a user is from remaining other users.

- **product_total_orders_delta_per_user** : difference between total number of orders placed for the product and total number of orders placed for the product by the specific user.

- **product_avg_add_to_cart_order_delta_per_user** : difference between product's average add to cart order based on all users and product's average add to cart order based on this specific users.

- **product_avg_order_dow_per_user** : difference between average day of week when the product is ordered based on all users and average day of week when the product is ordered based on this specifc user 
- **product_avg_order_hour_of_day_per_user**: difference between product's average hour of day when ordered and product's average hour of day when ordered by this user 
- **product_avg_days_since_prior_order_per_user**: difference between product's average days elapsed since last order placed and average days elapsed since last order placed by specifc user

In [16]:
df_X['product_total_orders_delta_per_user'] = df_X['product_total_orders'] - df_X['user_product_total_orders']

df_X['product_avg_add_to_cart_order_delta_per_user'] = df_X['product_avg_add_to_cart_order'] - \
                                                            df_X['user_product_avg_add_to_cart_order']

df_X['product_avg_order_dow_per_user'] = df_X['product_avg_order_dow'] - df_X['user_product_avg_order_dow']

df_X['product_avg_order_hour_of_day_per_user'] = df_X['product_avg_order_hour_of_day'] - \
                                                            df_X['user_product_avg_order_hour_of_day']

df_X['product_avg_days_since_prior_order_per_user'] = df_X['product_avg_days_since_prior_order'] - \
                                                            df_X['user_product_avg_days_since_prior_order']

In [17]:
df_X.head(2)

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day,latest_cart,in_cart,product_total_orders,...,user_total_products,user_avg_days_since_prior_order,user_avg_order_dow,user_avg_order_hour_of_day,user_product_order_freq,product_total_orders_delta_per_user,product_avg_add_to_cart_order_delta_per_user,product_avg_order_dow_per_user,product_avg_order_hour_of_day_per_user,product_avg_days_since_prior_order_per_user
0,1,709,20.0,1,6.0,0.0,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,1852,...,135,7.070968,1.651042,19.333333,0.2,1851,-14.198164,2.776458,-7.761879,4.432725
1,196,709,25.0,2,8.0,0.5,21.0,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,35791,...,135,7.070968,1.651042,19.333333,0.4,35789,-21.278226,2.398550,-8.476041,3.971826


In [18]:
df_X.shape

(5935424, 26)

### One hot encoding of  variable Department

Obtaining department names for correspoding product ids. Followed by merging it with df_X. Next one hot encoding of the categorical variable (department) is obtained.



In [19]:
prod_dept_df = products_df.merge(departments_df, on = 'department_id')
prod_dept_df = prod_dept_df[['product_id', 'department']]
prod_dept_df.head()

,product_id,department
0,1,snacks
1,16,snacks
2,25,snacks
3,32,snacks
4,41,snacks


In [20]:
df_X = df_X.merge(prod_dept_df, on = 'product_id')
df_X = df_X.dropna()
df_X = pd.concat([df_X, pd.get_dummies(df_X['department'])], axis=1)
df_X.head()

,product_id,user_id,user_product_avg_add_to_cart_order,user_product_total_orders,user_product_avg_days_since_prior_order,user_product_avg_order_dow,user_product_avg_order_hour_of_day,latest_cart,in_cart,product_total_orders,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
0,1,709,20.000000,1,6.0,0.000000,21.000000,"{14177, 8859, 16068, 32005, 28577, 10279, 4544...",0,1852,...,0,0,0,0,0,0,0,0,0,1
1,1,777,7.000000,1,26.0,1.000000,7.000000,"{43352, 6184, 16797}",0,1852,...,0,0,0,0,0,0,0,0,0,1
2,1,1052,1.500000,2,17.0,1.000000,16.000000,{17207},0,1852,...,0,0,0,0,0,0,0,0,0,1
3,1,1494,12.666667,3,6.0,3.333333,8.666667,{44560},0,1852,...,0,0,0,0,0,0,0,0,0,1
4,1,2850,10.000000,1,9.0,1.000000,16.000000,"{35921, 5068, 10070, 12015}",0,1852,...,0,0,0,0,0,0,0,0,0,1


In [21]:
del df_X['department']

Finally df_X is a normalized product-user pair wherein every training user_id's previously ordered product along with other user level features, product level features and if the product is currently present in the user's latest order. This flat normalized data is exported as a csv which will be next used for training logistic regression in order to predict the products ordered by test users.

In [22]:
df_X.to_csv('flatfile_5june_complete.csv')